In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.size'] = 15
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = 16,8

from collections import Counter,defaultdict
from itertools import chain, combinations

import warnings
warnings.filterwarnings('ignore')

from itertools import chain

import re
from konlpy.tag import Okt, Mecab
import nltk
from nltk.corpus import stopwords


from collections import Counter
import pickle

import sent2vec
import hnswlib
from gensim.models import Word2Vec
from gensim import corpora
from scipy.sparse import *
from scipy.sparse.linalg import svds

# -------
with open('/home/ubuntu/workspace/jh/project/data/10_28_stopword.pickle','rb') as file:
    stopword = pickle.load(file)

m = Mecab('/home/ubuntu/workspace/mecab-ko-dic-2.1.1-20180720')

In [2]:
# konlpy_total을 통해 데이터를 통해서 sent2vec 모델을 생성 
train = pd.read_json('data/1108_new_train.json')

tag_title = pd.read_json('/home/ubuntu/workspace/jh/project/data/1108_konlpy_text.json')

train['tag'] = tag_title['tags']
train['new_tags__'] = train['new_tags'].apply(lambda x : " ".join(x))
train['tags__'] = train['tag'].apply(lambda x : " ".join(x))

ply_title = train['plylst_title'].tolist()
tags = train['tags__'].tolist()
new_tags = train['new_tags__'].tolist()


train.head(3)

,tags,id,new_tags,plylst_title,songs,like_cnt,updt_date,tag,new_tags__,tags__
0,[락],61281,"[여행, 락]",여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,[락],여행 락,락
1,"[추억, 회상]",10532,"[회상, 추억]",요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,"[추억, 회상]",회상 추억,추억 회상
2,"[까페, 잔잔한]",76951,"[편하, 까페, 잔잔]","편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,"[까페, 잔잔]",편하 까페 잔잔,까페 잔잔


# 1. Sent2Vec

- word2vec는 단어 단위에서는 의미가 있지만 서브샘플링과 다이나믹 컨제스트 윈도우, n_그램의 영향으로 문장 단위로 확장하면 해석이 힘들수가 있다
- 그렇기 때문에 word2vec에서 CBOW를 확장한 Sent2vec을 사용
    - word2vec과 달리 서브샘플링과 다이나믹 컨제스트 윈도우를 비활성화해서 문장 전체의 의미를 살릴 수 있다
    - n_그램 또한 전산 언어학에서의 n_그램이 아닌 비아그램의 최대거리를 뜻하며 n_그램의 확장 또한 뒤로 진행하고 이전의 단어는 포함하지 않아 문장의 의미를 살릴 수 있다

[sent2vec doc](https://github.com/epfml/sent2vec)

다운 방법은 위의 링크에서 확인 가능

## 1_1. sent2vec 모델 학습 

- 모델을 학습시킬 경우 아래와 같이 txt파일로 진행하면 편하게 진행 가능하다
- 또한 sent2vec은 C++언어를 통해 학습을 진행하기 때문에 해당 폴더에서 학습을 진행 시킨다
- 그렇기 때문에 아래와 같이 경로를 지정한 후 os.system()을 통해 학습을 진행하고 바로 모델을 저장한다

In [19]:
s2v_text = train['new_tags__'].tolist()

f = open("./data/s2v_text.txt", 'w')
for i in s2v_text:
    x = f'{i} \n'
    f.write(x)

f.close

<function TextIOWrapper.close()>

In [38]:
cmd = "/home/ubuntu/workspace/sent2vec/fasttext sent2vec" 
cmd += " -input /home/ubuntu/workspace/jh/project/data/s2v_text.txt" 
cmd += " -output ./model/s2v_model" 
cmd += " -minCount 0" 
cmd += " -dim 100" 
cmd += " -epoch 20" 
cmd += " -lr 0.2" 
cmd += " -wordNgrams 3" 
cmd += " -loss ns" 
cmd += " -neg 11" 
cmd += " -thread 16" 
cmd += " -t 0.000005" 
cmd += " -dropoutK 4" 
cmd += " -minCountLabel 20" 
cmd += " -bucket 100000" 
cmd += " -maxVocabSize 20000" 
cmd += " -numCheckPoints 1"

In [39]:
import os 
os.system(cmd)

Read 0M words
Number of words:  9303
Number of labels: 0
Progress: 100.0%  words/sec/thread: 504460  lr: 0.000000  loss: 2.103966  eta: 0h0m 


0

# 2. KNN

- sent2vec의 모델 학습은 이전 파일인 __3_1.Word2Vec_Knn_model_make.ipynb의 방식과 똑같은 방식으로 진행한다

In [40]:
s2v_model = sent2vec.Sent2vecModel()
s2v_model.load_model('/home/ubuntu/workspace/jh/project/model/s2v_model.bin')

## 2_1 KNN 모델 학습을 위한 전처리

In [41]:
# title
title_emb = s2v_model.embed_sentences(ply_title)
title_idx_to_emb = {}
for idx, emb in enumerate(title_emb):
    if emb.sum() != 0:
        title_idx_to_emb[idx] = emb

# tags
tags_emb = s2v_model.embed_sentences(tags)
tags_idx_to_emb = {}
for idx, emb in enumerate(tags_emb):
    if emb.sum() != 0:
        tags_idx_to_emb[idx] = emb

# new_tags
new_tags_emb = s2v_model.embed_sentences(new_tags)
new_tags_idx_to_emb = {}
for idx, emb in enumerate(new_tags_emb):
    if emb.sum() != 0:
        new_tags_idx_to_emb[idx] = emb

## 2_2 knn 모델 생성


In [42]:
sp = 'cosine'
dim = s2v_model.get_emb_size()
    
# title
p_title = hnswlib.Index(space = sp, dim = dim)
p_title.init_index(max_elements = len(title_idx_to_emb), 
                   ef_construction = 100, M = 16, random_seed = 100)
p_title.add_items(list(title_idx_to_emb.values()), list(title_idx_to_emb.keys()))

# tags
p_tags = hnswlib.Index(space = sp, dim = dim)
p_tags.init_index(max_elements = len(tags_idx_to_emb), 
                  ef_construction = 100, M = 16, random_seed = 100)
p_tags.add_items(list(tags_idx_to_emb.values()), list(tags_idx_to_emb.keys()))

# new_tags
p_new_tags = hnswlib.Index(space = sp, dim = dim)
p_new_tags.init_index(max_elements = len(new_tags_idx_to_emb), 
                      ef_construction = 100, M = 16, random_seed = 100)
p_new_tags.add_items(list(new_tags_idx_to_emb.values()), list(new_tags_idx_to_emb.keys()))

## 2_3. 모델 저장

In [43]:
# 생성된 모델 pickle 형태로 변환
pickle.dump(p_title, open('model/p_title.pickle', 'wb') )
pickle.dump(p_tags, open('model/p_tags.pickle', 'wb') )
pickle.dump(p_new_tags, open('model/p_new_tags.pickle', 'wb') )